In [238]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from helper import *
from sklearn.preprocessing import MultiLabelBinarizer
from ast import literal_eval

In [239]:
# Assuming you have a DataFrame named df with the mentioned columns
file_path_HLES = '../Dataset/SurveyData/SurveyData_CSV_DAP_2021_HLES_dog_owner_v1.0.csv'
df_HLES = pd.read_csv(file_path_HLES)
df_HLES

/var/folders/zd/3_6frj8s08b9zb1f3y0xqr2r0000gn/T/ipykernel_94793/3747774317.py:3: DtypeWarning: Columns (5,21,75,77,79,81,83,85,87,89,191,192,193,194,195,196,197,198,199,200,201,204,214,285,295,516,566) have mixed types. Specify dtype option on import or set low_memory=False.
  df_HLES = pd.read_csv(file_path_HLES)


,dog_id,owner_id,st_hles_completion_date,dd_breed_pure_or_mixed,dd_breed_pure,dd_breed_pure_non_akc,dd_breed_mixed_primary,dd_breed_mixed_secondary,dd_age_years,dd_age_basis,...,oc_primary_residence_state,oc_primary_residence_census_division,oc_primary_residence_ownership,oc_primary_residence_ownership_other_description,oc_primary_residence_time_percentage,oc_secondary_residence,oc_secondary_residence_state,oc_secondary_residence_ownership,oc_secondary_residence_ownership_other_description,oc_secondary_residence_time_percentage
0,10509,14660,2020-10-17,2,NaN,NaN,16.0,999.0,16.333333,2,...,MO,4.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
1,74227,16755,2020-10-26,2,NaN,NaN,133.0,12.0,1.833333,1,...,MD,5.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
2,32487,8568,2020-09-07,1,159.0,NaN,NaN,NaN,13.000000,3,...,VA,5.0,1,NaN,NaN,False,NaN,NaN,NaN,NaN
3,63150,5380,2020-08-21,2,NaN,NaN,999.0,999.0,13.000000,3,...,CA,9.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
4,33369,15957,2020-10-23,2,NaN,NaN,230.0,82.0,2.166667,1,...,OH,3.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,32736,2021-06-22,2,NaN,NaN,276.0,1.0,10.000000,3,...,CA,9.0,2,NaN,5.0,True,NH,2.0,NaN,1.0
33168,33783,9482,2020-09-15,2,NaN,NaN,130.0,195.0,3.000000,3,...,WA,9.0,1,NaN,NaN,False,NaN,NaN,NaN,NaN
33169,98862,32312,2021-05-25,2,NaN,NaN,60.0,49.0,4.000000,3,...,AR,7.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
33170,99462,32692,2021-06-18,2,NaN,NaN,204.0,27.0,0.916667,1,...,WA,9.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN


In [240]:
df_HLES['dd_breed_pure'].value_counts()

dd_breed_pure
153.0    1985
120.0    1728
114.0     759
195.0     547
18.0      543
         ... 
198.0       1
58.0        1
170.0       1
35.0        1
178.0       1
Name: count, Length: 248, dtype: int64

In [241]:
# Map the classification dict in helper.py to the 'Breed' column
df_HLES['dd_breed_mixed_primary'] = df_HLES['dd_breed_mixed_primary'].map(lambda x: pure_breed_mapping.get(x, x))
df_HLES['dd_breed_mixed_secondary'] = df_HLES['dd_breed_mixed_secondary'].map(lambda x: pure_breed_mapping.get(x, x))
df_HLES['dd_breed_pure'] = df_HLES['dd_breed_pure'].map(lambda x: pure_breed_mapping.get(x, x))
df_HLES[['dog_id','dd_breed_pure_non_akc', 'dd_breed_pure' ,'dd_breed_mixed_primary', 'dd_breed_mixed_secondary']]

,dog_id,dd_breed_pure_non_akc,dd_breed_pure,dd_breed_mixed_primary,dd_breed_mixed_secondary
0,10509,NaN,NaN,Australian Cattle Dog,Unknown
1,74227,NaN,NaN,Ibizan Hound,American Staffordshire Terrier
2,32487,NaN,Lhasa Apso,NaN,NaN
3,63150,NaN,NaN,Unknown,Unknown
4,33369,NaN,NaN,Siberian Husky,Collie
...,...,...,...,...,...
33167,91849,NaN,NaN,Yorkshire Terrier,Affenpinscher
33168,33783,NaN,NaN,Havanese,Poodle
33169,98862,NaN,NaN,Bull Terrier,Boston Terrier
33170,99462,NaN,NaN,Pug,Beagle


In [242]:
condition = df_HLES['dd_breed_pure'] == "My dog is a non-AKC purebred not listed here"
df_HLES.loc[condition, 'dd_breed_pure'] = df_HLES.loc[condition, 'dd_breed_pure_non_akc'].values
df_HLES.loc[condition, 'dd_breed_pure'] = df_HLES.loc[condition, 'dd_breed_pure'].map(lambda x: classification_mapping_2.get(x, x) if x in classification_mapping_2 else x)
df_HLES[['dd_breed_pure', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary']]

,dd_breed_pure,dd_breed_mixed_primary,dd_breed_mixed_secondary
0,NaN,Australian Cattle Dog,Unknown
1,NaN,Ibizan Hound,American Staffordshire Terrier
2,Lhasa Apso,NaN,NaN
3,NaN,Unknown,Unknown
4,NaN,Siberian Husky,Collie
...,...,...,...
33167,NaN,Yorkshire Terrier,Affenpinscher
33168,NaN,Havanese,Poodle
33169,NaN,Bull Terrier,Boston Terrier
33170,NaN,Pug,Beagle


In [243]:
# Combine the values from the three breed columns into a single series
combined_breeds = pd.concat([df_HLES['dd_breed_pure'], df_HLES['dd_breed_mixed_primary'], df_HLES['dd_breed_mixed_secondary']])
combined_breeds

0                   NaN
1                   NaN
2            Lhasa Apso
3                   NaN
4                   NaN
              ...      
33167     Affenpinscher
33168            Poodle
33169    Boston Terrier
33170            Beagle
33171           Unknown
Length: 99516, dtype: object

In [244]:
combined_breeds.value_counts()

Labrador Retriever               5654
Unknown                          3628
Golden Retriever                 3115
Poodle                           2644
German Shepherd Dog              2261
                                 ... 
Azawakh                             1
PODENCO                             1
Large Muensterlaender               1
Bavarian Mountain Scent Hound       1
Croatian Sheepdog                   1
Name: count, Length: 302, dtype: int64

In [245]:
# Define the number of most common values to keep
top_n_values = 50

# Identify the 50 most common values in the column
top_values = combined_breeds.value_counts().head(top_n_values)
top_values

Labrador Retriever                5654
Unknown                           3628
Golden Retriever                  3115
Poodle                            2644
German Shepherd Dog               2261
Border Collie                     1584
Chihuahua                         1509
American Pitbull Terrier          1437
Australian Shepherd               1375
Beagle                            1034
Australian Cattle Dog             1030
Dachshund                          951
Boxer                              946
American Staffordshire Terrier     923
Siberian Husky                     868
Jack Russell Terrier               715
Shih Tzu                           664
Poodle (Toy)                       566
Yorkshire Terrier                  553
Cocker Spaniel                     519
Miniature Schnauzer                470
Great Pyrenees                     467
Maltese                            453
Pug                                435
Rottweiler                         393
Great Dane               

In [246]:
df_HLES['dd_breed_pure'].fillna('', inplace=True)
df_HLES['dd_breed_mixed_primary'].fillna('', inplace=True)
df_HLES['dd_breed_mixed_secondary'].fillna('', inplace=True)

# Combine the breed columns into a single column for each row
df_HLES['breeds'] = df_HLES.apply(lambda row: [row['dd_breed_pure'], row['dd_breed_mixed_primary'], row['dd_breed_mixed_secondary']], axis=1)
df_HLES['breeds']

0                      [, Australian Cattle Dog, Unknown]
1        [, Ibizan Hound, American Staffordshire Terrier]
2                                        [Lhasa Apso, , ]
3                                    [, Unknown, Unknown]
4                              [, Siberian Husky, Collie]
                               ...                       
33167                [, Yorkshire Terrier, Affenpinscher]
33168                                [, Havanese, Poodle]
33169                    [, Bull Terrier, Boston Terrier]
33170                                     [, Pug, Beagle]
33171               [, Treeing Walker Coonhound, Unknown]
Name: breeds, Length: 33172, dtype: object

In [247]:
def classify_breed(weight_class):
    if weight_class in [1, 2]:
        return ['Mixed Breed Small (0 - 20lbs)']
    elif weight_class in [3, 4, 5, 6]:
        return ['Mixed Breed Medium (21 - 60lbs)']
    elif weight_class in [7, 8, 9, 10, 11]:
        return ['Mixed Breed Large (61lbs+)']
    else:
        return ['Unknown Mix']


# Assuming df_HLES is your dataframe
mask_unknown_breed = (df_HLES['dd_breed_mixed_primary'] == 'Unknown') & (df_HLES['dd_breed_mixed_secondary'] == 'Unknown')

df_HLES.loc[mask_unknown_breed, 'breeds'] = df_HLES.loc[mask_unknown_breed, 'dd_weight_range'].apply(classify_breed)

In [248]:
df_HLES.loc[mask_unknown_breed, 'breeds'].value_counts()

breeds
[Mixed Breed Medium (21 - 60lbs)]    335
[Mixed Breed Large (61lbs+)]          76
[Mixed Breed Small (0 - 20lbs)]       64
Name: count, dtype: int64

In [249]:
# Create a MultiLabelBinarizer to one-hot encode the breeds
mlb = MultiLabelBinarizer()
mlb_df = mlb.fit_transform(df_HLES['breeds'])
mlb_df

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [250]:
mlb.classes_

array(['', 'Affenpinscher', 'Afghan Hound', 'African Village Dog',
       'Airedale Terrier', 'Akbash', 'Akita', 'Alaskan Klee Kai',
       'Alaskan Malamute', 'American Bulldog', 'American Bully',
       'American English (aka Redtick) Coonhound',
       'American English Coonhound', 'American Eskimo Dog',
       'American Foxhound', 'American Hairless Terrier',
       'American Indian Dog', 'American Leopard Hound',
       'American Pitbull Terrier', 'American Staffordshire Terrier',
       'American Water Spaniel', 'Anatolian Shepherd Dog',
       'Appenzeller Sennenhund', 'Aussie - Dor', 'Australian Cattle Dog',
       'Australian Kelpie', 'Australian Labradoodle',
       'Australian Shepherd', 'Australian Stumpy Tail Cattle Dog',
       'Australian Terrier', 'Azawakh', 'Barbet', 'Basenji',
       'Basset Fauve de Bretagne', 'Basset Hound',
       'Bavarian Mountain Scent Hound', 'Beagle', 'Bearded Collie',
       'Beauceron', 'Bedlington Terrier', 'Belgian Laekenois',
       'Belg

In [251]:
# Fit and transform the 'breeds' column
breed_encoded = pd.DataFrame(mlb.fit_transform(df_HLES['breeds']), columns=[f'breeds_{col}' for col in mlb.classes_])
breed_encoded.drop(['breeds_', 'breeds_Unknown'], axis=1, inplace=True)

# Concatenate the one-hot encoded breeds with the original dataframe
df = pd.concat([df_HLES[['dog_id', 'dd_breed_pure', 'dd_breed_mixed_primary', 'dd_breed_mixed_secondary']], breed_encoded], axis=1)
df

,dog_id,dd_breed_pure,dd_breed_mixed_primary,dd_breed_mixed_secondary,breeds_Affenpinscher,breeds_Afghan Hound,breeds_African Village Dog,breeds_Airedale Terrier,breeds_Akbash,breeds_Akita,...,breeds_Whippet,breeds_Windsprite,breeds_Wire Fox Terrier,breeds_Wirehaired Pointing Griffon,breeds_Wirehaired Vizsla,breeds_Wolf-hybrid,breeds_Working Kelpie,breeds_Xoloitzcuintli,breeds_Yorkshire Terrier,breeds_shiloh shepherd
0,10509,,Australian Cattle Dog,Unknown,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,74227,,Ibizan Hound,American Staffordshire Terrier,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,32487,Lhasa Apso,,,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,63150,,Unknown,Unknown,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33369,,Siberian Husky,Collie,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,,Yorkshire Terrier,Affenpinscher,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
33168,33783,,Havanese,Poodle,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33169,98862,,Bull Terrier,Boston Terrier,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33170,99462,,Pug,Beagle,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [252]:
# Concatenate the one-hot encoded breeds with the original dataframe
id_breed_onehot = pd.concat([df_HLES[['dog_id']], breed_encoded], axis=1)
id_breed_onehot

,dog_id,breeds_Affenpinscher,breeds_Afghan Hound,breeds_African Village Dog,breeds_Airedale Terrier,breeds_Akbash,breeds_Akita,breeds_Alaskan Klee Kai,breeds_Alaskan Malamute,breeds_American Bulldog,...,breeds_Whippet,breeds_Windsprite,breeds_Wire Fox Terrier,breeds_Wirehaired Pointing Griffon,breeds_Wirehaired Vizsla,breeds_Wolf-hybrid,breeds_Working Kelpie,breeds_Xoloitzcuintli,breeds_Yorkshire Terrier,breeds_shiloh shepherd
0,10509,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,74227,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,32487,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,63150,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
33168,33783,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33169,98862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33170,99462,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [219]:
id_breed_onehot.to_pickle('features/breed_v2.pkl')
id_breed_onehot.to_csv('features/breed_v2.csv', index=False)

In [253]:
# Combine the values from the three breed columns into a single series
combined_breeds = pd.concat([df_HLES['dd_breed_pure'], df_HLES['dd_breed_mixed_primary'], df_HLES['dd_breed_mixed_secondary']])

# Define the number of most common values to keep
top_n_values = 51

# Identify the top 50 most common values in the column
top_values = combined_breeds.value_counts().head(top_n_values).index

# Fill NaN values with an empty string in the breed columns
df_HLES['dd_breed_pure'].fillna('', inplace=True)
df_HLES['dd_breed_mixed_primary'].fillna('', inplace=True)
df_HLES['dd_breed_mixed_secondary'].fillna('', inplace=True)

# Replace empty strings with a placeholder value
#df_HLES['dd_breed_pure'].replace('', 'Unknown', inplace=True)
#df_HLES['dd_breed_mixed_primary'].replace('', 'Unknown', inplace=True)
#df_HLES['dd_breed_mixed_secondary'].replace('', 'Unknown', inplace=True)

# Filter the breeds to include only the top 50 breeds
df_HLES['dd_breed_pure'] = df_HLES['dd_breed_pure'].apply(lambda x: x if x in top_values else 'Other')
df_HLES['dd_breed_mixed_primary'] = df_HLES['dd_breed_mixed_primary'].apply(lambda x: x if x in top_values else 'Other')
df_HLES['dd_breed_mixed_secondary'] = df_HLES['dd_breed_mixed_secondary'].apply(lambda x: x if x in top_values else 'Other')

# Combine the breed columns into a single column for each row
df_HLES['breeds'] = df_HLES.apply(lambda row: [row['dd_breed_pure'], row['dd_breed_mixed_primary'], row['dd_breed_mixed_secondary']], axis=1)

# Create a MultiLabelBinarizer to one-hot encode the breeds
mlb = MultiLabelBinarizer()
breed_encoded = pd.DataFrame(mlb.fit_transform(df_HLES['breeds']), columns=[f'breeds_{col}' for col in mlb.classes_])
breed_encoded.drop(['breeds_', 'breeds_Unknown'], axis=1, inplace=True)

# Concatenate the one-hot encoded breeds with the original dataframe
id_breed_onehot_top50 = pd.concat([df_HLES[['dog_id']], breed_encoded], axis=1)
id_breed_onehot_top50

,dog_id,breeds_American Pitbull Terrier,breeds_American Staffordshire Terrier,breeds_Australian Cattle Dog,breeds_Australian Shepherd,breeds_Basset Hound,breeds_Beagle,breeds_Bernese Mountain Dog,breeds_Bichon Frise,breeds_Border Collie,...,breeds_Poodle (Toy),breeds_Pug,breeds_Rat Terrier,breeds_Rhodesian Ridgeback,breeds_Rottweiler,breeds_Shetland Sheepdog,breeds_Shih Tzu,breeds_Siberian Husky,breeds_West Highland White Terrier,breeds_Yorkshire Terrier
0,10509,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,74227,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,32487,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,63150,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33369,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
33168,33783,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33169,98862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33170,99462,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [254]:
id_breed_onehot_top50.to_pickle('features/breed_top50_v2.pkl')
id_breed_onehot_top50.to_csv('features/breed_top50_v2.csv', index=False)

## Breed pure or mixed

In [255]:
# dd_breed_pure_or_mixed
# 2 -> mixed
# 1 -> pure

df_HLES

,dog_id,owner_id,st_hles_completion_date,dd_breed_pure_or_mixed,dd_breed_pure,dd_breed_pure_non_akc,dd_breed_mixed_primary,dd_breed_mixed_secondary,dd_age_years,dd_age_basis,...,oc_primary_residence_census_division,oc_primary_residence_ownership,oc_primary_residence_ownership_other_description,oc_primary_residence_time_percentage,oc_secondary_residence,oc_secondary_residence_state,oc_secondary_residence_ownership,oc_secondary_residence_ownership_other_description,oc_secondary_residence_time_percentage,breeds
0,10509,14660,2020-10-17,2,,NaN,Australian Cattle Dog,Unknown,16.333333,2,...,4.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN,"[, Australian Cattle Dog, Unknown]"
1,74227,16755,2020-10-26,2,,NaN,Other,American Staffordshire Terrier,1.833333,1,...,5.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN,"[, Other, American Staffordshire Terrier]"
2,32487,8568,2020-09-07,1,Other,NaN,,,13.000000,3,...,5.0,1,NaN,NaN,False,NaN,NaN,NaN,NaN,"[Other, , ]"
3,63150,5380,2020-08-21,2,,NaN,Unknown,Unknown,13.000000,3,...,9.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN,"[, Unknown, Unknown]"
4,33369,15957,2020-10-23,2,,NaN,Siberian Husky,Collie,2.166667,1,...,3.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN,"[, Siberian Husky, Collie]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33167,91849,32736,2021-06-22,2,,NaN,Yorkshire Terrier,Other,10.000000,3,...,9.0,2,NaN,5.0,True,NH,2.0,NaN,1.0,"[, Yorkshire Terrier, Other]"
33168,33783,9482,2020-09-15,2,,NaN,Havanese,Poodle,3.000000,3,...,9.0,1,NaN,NaN,False,NaN,NaN,NaN,NaN,"[, Havanese, Poodle]"
33169,98862,32312,2021-05-25,2,,NaN,Other,Boston Terrier,4.000000,3,...,7.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN,"[, Other, Boston Terrier]"
33170,99462,32692,2021-06-18,2,,NaN,Pug,Beagle,0.916667,1,...,9.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN,"[, Pug, Beagle]"


In [256]:
df_HLES['dd_breed_pure_or_mixed'].value_counts()

dd_breed_pure_or_mixed
1    16619
2    16553
Name: count, dtype: int64

In [257]:
df_HLES['dd_breed_pure_or_mixed'].replace({2: 1, 1: 0}, inplace=True)

In [258]:
id_pure_or_mixed = df_HLES[['dog_id', 'dd_breed_pure_or_mixed']]
id_pure_or_mixed

,dog_id,dd_breed_pure_or_mixed
0,10509,1
1,74227,1
2,32487,0
3,63150,1
4,33369,1
...,...,...
33167,91849,1
33168,33783,1
33169,98862,1
33170,99462,1


In [260]:
id_pure_or_mixed.to_pickle('features/breed_pure_or_mix.pkl')
id_pure_or_mixed.to_csv('features/breed_pure_or_mix.csv', index=False)